In [1]:
! pip install fastai==0.7.0
! pip install torchtext==0.2.3

    100% |████████████████████████████████| 122kB 3.8MB/s 
    100% |████████████████████████████████| 2.2MB 13.7MB/s 
    100% |████████████████████████████████| 71kB 20.9MB/s 
    100% |████████████████████████████████| 122kB 31.2MB/s 
    100% |████████████████████████████████| 1.5MB 15.4MB/s 
    100% |████████████████████████████████| 184kB 28.2MB/s 
    100% |████████████████████████████████| 61kB 25.4MB/s 
    100% |████████████████████████████████| 112kB 28.7MB/s 
    100% |████████████████████████████████| 3.6MB 10.7MB/s 
    100% |████████████████████████████████| 496.4MB 29kB/s 
    100% |████████████████████████████████| 122kB 29.6MB/s 
    100% |████████████████████████████████| 11.6MB 3.2MB/s 
    100% |████████████████████████████████| 92kB 22.2MB/s 
    100% |████████████████████████████████| 61kB 19.5MB/s 
    100% |████████████████████████████████| 81kB 24.1MB/s 
  Running setup.py bdist_wheel for feather-format ... - done
  Stored in directory: /root/.cache/pip/whe

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

In [0]:
PATH='data/nietzsche/'

In [5]:
get_data("https://s3.amazonaws.com/text-datasets/nietzsche.txt", f'{PATH}nietzsche.txt')
text = open(f'{PATH}nietzsche.txt').read()
print('corpus length:', len(text))

nietzsche.txt: 606kB [00:01, 355kB/s]                            

corpus length: 600893


In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 85


In [7]:
chars.insert(0, "\0")

''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxy'

In [0]:
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}

In [9]:
idx = [char_indices[c] for c in text]

idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [10]:
from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *

PATH='/content/data/nietzsche/'

TRN_PATH = 'TRN/'
VAL_PATH = 'VAL/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

# Note: The student needs to practice her shell skills and prepare her own dataset before proceeding:
# - trn/trn.txt (first 80% of nietzsche.txt)
# - val/val.txt (last 20% of nietzsche.txt)

%ls {PATH}

nietzsche.txt


In [18]:
ls 

nietzsche.txt  TRN/  trn.txt  VAL/  val.txt


In [12]:
%cd  data/nietzsche/



/content/data/nietzsche


In [0]:
!mkdir VAL
!mkdir TRN

In [0]:
f= open("trn.txt","w+")
for i in range(len(text[:530000])):
     f.write(text[i])
f.close() 

f= open("val.txt","w+")
for i in range(len(text[530000:])):
     f.write(text[i])
f.close() 

In [19]:
# ! wc -m  val.txt
! mv -v /content/data/nietzsche/val.txt /content/data/nietzsche/VAL/
! mv -v /content/data/nietzsche/trn.txt /content/data/nietzsche/TRN/

renamed '/content/data/nietzsche/val.txt' -> '/content/data/nietzsche/VAL/val.txt'
renamed '/content/data/nietzsche/trn.txt' -> '/content/data/nietzsche/TRN/trn.txt'


In [41]:
TEXT = data.Field(lower=True, tokenize=list)
bs=64; bptt=50; n_fac=100; n_hidden=1024

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(161, 55, 1, 521287)

In [0]:
# 512*2

In [0]:
from fastai import sgdr

n_hidden=1024

In [0]:
class CharSeqStatefulLSTM(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.LSTM(n_fac, n_hidden, nl, dropout=0.5)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden)))

In [0]:
m = CharSeqStatefulLSTM(md.nt, n_fac, 512, 3).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-5)

In [0]:
os.makedirs(f'{PATH}models', exist_ok=True)

In [46]:
fit(m, md, 2, lo.opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      3.058062   3.044033  
    1      3.041689   3.044638  


[array([3.04464])]

In [47]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**4-1, lo.opt, F.nll_loss, callbacks=cb)

epoch      trn_loss   val_loss   
    0      3.016774   2.989205  
    1      3.004507   2.997172  
    2      2.940385   2.875598  
    3      3.025433   3.003709  
    4      2.844473   2.683357  
    5      2.066826   1.826603  
    6      1.743862   1.643534  
    7      1.852061   1.697437  
    8      1.712708   1.567653  
    9      1.605037   1.470101  
    10     1.517391   1.40102   
    11     1.4405     1.334305  
    12     1.372975   1.280644  
    13     1.3177     1.238514  
    14     1.290842   1.226868  


[array([1.22687])]

In [48]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**6-1, lo.opt, F.nll_loss, callbacks=cb)

epoch      trn_loss   val_loss   
    0      1.44048    1.313112  
    1      1.488313   1.350004  
    2      1.36552    1.267718  
    3      1.488569   1.367849  
    4      1.425236   1.302421  
    5      1.334832   1.226359  
    6      1.267407   1.196797  
    7      1.476726   1.369066  
    8      1.456075   1.341615  
    9      1.408075   1.300758  
    10     1.357125   1.252813  
    11     1.306475   1.204362  
    12     1.254089   1.154594  
    13     1.202228   1.122787  
    14     1.17207    1.11179   
    15     1.457128   1.349205  
    16     1.436052   1.320655  
    17     1.41067    1.296875  
    18     1.390619   1.289492  
    19     1.368401   1.262586  
    20     1.34629    1.249168  
    21     1.319573   1.218542  
    22     1.295882   1.193858  
    23     1.264732   1.168474  
    24     1.235027   1.142553  
    25     1.197942   1.112822  
    26     1.163695   1.076679  
    27     1.127845   1.043957  
    28     1.093791   1.015454  
    29   

[array([0.82454])]

In [0]:
def get_next(inp):
    idxs = TEXT.numericalize(inp)
    p = m(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]

In [0]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [51]:
print(get_next_n('for thos', 400))

for those bringing. it betrays (happinessimpally, requirement, down unduchmile for it (would lie fol actions! what isworda wrings. in case gives himself and begin to it. the proper rus! it may belief in the dreads to the tow, somethings at things; itbestin at otherwism. does not some new [look at a distracting--it was that attempters; such arrangement, shame-posterity insomuch as ever of feiting upon the 
